In [18]:
import os
import sys
import urllib.request
import datetime
import time
import json
client_id= ''
client_secret= ''
#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [19]:
#[CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start,display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)


In [20]:
#[CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,
    'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return

def getPostData_Blog(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['bloggerlink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['postdate'], '%Y%m%d')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,
    'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return
# "title"
#"description"
# "link":"
# "bloggername":"하루집사의 IT한마당",
# "bloggerlink":"blog.naver.com\/dltmdgns2525",
# "postdate":"20250218"


In [21]:
#[CODE 0]
def main():
    node = 'blog' #크롤링할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []
    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
    total = jsonResponse['total']
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            # getPostData(post, jsonResult, cnt) #[CODE 3]
            getPostData_Blog(post, jsonResult, cnt) #[CODE 3]
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()

[2025-02-19 13:59:03.635964] Url Request Success
[2025-02-19 13:59:03.925179] Url Request Success
[2025-02-19 13:59:04.244199] Url Request Success
[2025-02-19 13:59:04.541219] Url Request Success
[2025-02-19 13:59:04.824432] Url Request Success
[2025-02-19 13:59:05.114994] Url Request Success
[2025-02-19 13:59:05.406112] Url Request Success
[2025-02-19 13:59:05.713545] Url Request Success
[2025-02-19 13:59:06.111345] Url Request Success
[2025-02-19 13:59:06.410692] Url Request Success
HTTP Error 400: Bad Request
[2025-02-19 13:59:06.493206] Error for URL : https://openapi.naver.com/v1/search/blog.json?query=%EC%B9%98%EB%A7%A5&start=1001&display=100
전체 검색 : 1123275 건
가져온 데이터 : 1000 건
치맥_naver_blog.json SAVED
